## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

## A1.  


**"42f05b9372a9a4a470db3b52817899b99a76ee73"**

#### Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```
## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

## A2.

**'index'**.

## Q3. Searching

Now let's search in our index. 

For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result? 

Use only `question` and `text` fields and give `question` a boost of 4

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

## A3.


**74.05**

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client = Elasticsearch('http://localhost:9200') 

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questionss"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questionss'})

In [8]:
from tqdm.auto import tqdm

c:\Users\Odiaka\miniconda3\envs\alexey\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:07<00:00, 133.00it/s]


In [24]:
query = 'How do I execute a command in a running docker container?'

In [25]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [26]:
elastic_search(query)

[75.54128, 43.922554, 38.684105, 38.33403, 35.94081]

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

## A4.


**'How do I copy files from a different folder into docker container’s working directory?'**

In [27]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [28]:
elastic_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. Let's use these templates:


```python
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

Separate context entries by two linebreaks (`\n\n`)

Put the data from Q4 there. 

What's the length of the result? (use the `len` function)

* 962
* 1462
* 1962
* 2462

## A5.


**1462**.

In [29]:
data = elastic_search(query)

In [30]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [31]:
# Create context entries
context_entries = [context_template.format(question=entry['question'], text=entry['text']) for entry in data]

# Combine context entries with two line breaks between them
combined_context = "\n\n".join(context_entries).strip()
combined_context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [32]:
main_question = query

# Insert combined context into the prompt template
result = prompt_template.format(question=main_question, context=combined_context)

# Calculate the length of the result
length_of_result = len(result)

In [33]:
print(combined_context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [34]:
print(f"\nLength of result: {length_of_result}")


Length of result: 1462


In [35]:
print("\nFinal result:")
print(result)


Final result:
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_director

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

## A6.


**322**.

In [58]:
import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")

# Tokenize the final prompt
input_tokens = encoding.encode(result)

# Calculate the number of tokens
num_input_tokens = len(input_tokens)

# Print the number of tokens
print(f"Number of tokens in the prompt: {num_input_tokens}")

Number of tokens in the prompt: 322


322

In [61]:
output_tokens = encoding.decode_single_token_bytes(63842)

In [63]:
# Calculate the number of tokens
num_output_tokens = len(output_tokens)

# Print the number of tokens
print(f"Number of output tokens: {num_output_tokens}")

Number of output tokens: 6


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

## Bonus Answer 1.


**'To execute a command in a running Docker container, follow these steps:\n\n1. First, find the container ID of the running container by using the `docker ps` command.\n   \n   ```shell\n   docker ps\n   ```\n\n2. Once you have the container ID, use the `docker exec` command to run the desired command in the container. For example, to start a bash shell in the container with the found container ID, use:\n\n   ```shell\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of your running container.'**

In [48]:
from dotenv import load_dotenv 

load_dotenv()

True

In [49]:
from openai import OpenAI

client = OpenAI()

In [50]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [52]:
answer = llm(result)
answer

'To execute a command in a running Docker container, follow these steps:\n\n1. First, find the container ID of the running container by using the `docker ps` command.\n   \n   ```shell\n   docker ps\n   ```\n\n2. Once you have the container ID, use the `docker exec` command to run the desired command in the container. For example, to start a bash shell in the container with the found container ID, use:\n\n   ```shell\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of your running container.'

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

## Bonus Answer 2.


To calculate the cost of running 1000 requests with the given parameters, I'll use the pricing for the GPT-4o model:

Input cost: $5 per 1,000 tokens
Output cost: $15 per 1,000 tokens
Given:

Average tokens sent per request (input): 150
Average tokens received per request (output): 250
Let's break down the costs:

* 1. Calculate the Total Number of Tokens
For 1000 requests:

Total input tokens = 150 tokens/request * 1000 requests = 150,000 tokens
Total output tokens = 250 tokens/request * 1000 requests = 250,000 tokens
* 2. Calculate the Cost
Convert tokens to thousands (k):

Input tokens in thousands = 150,000 tokens / 1,000 = 150k tokens
Output tokens in thousands = 250,000 tokens / 1,000 = 250k tokens


Cost Calculation:

Input cost = 150k tokens * $0.005 = $750
Output cost = 250k tokens * $0.015 = $3,750
* 3. Total Cost
Total cost = Input cost + Output cost
Total cost = $750 + $3,750 = $4,500

Therefore, the cost to run 1000 requests with the given parameters will be $4,500 or roughly N6,707,250 (6/18/2024).




For Q6 ad Q7:

In [60]:
input_tokens_per_request = 322  # Tokens from Q6
output_tokens_per_request = 7  # Tokens from Q7
num_requests = 1000
input_cost_per_1k = 0.005  # $0.005 per 1K tokens
output_cost_per_1k = 0.015  # $0.015 per 1K tokens

def calculate_cost(input_tokens_per_request, output_tokens_per_request, num_requests, input_cost_per_1k, output_cost_per_1k):
    total_input_tokens = input_tokens_per_request * num_requests
    total_output_tokens = output_tokens_per_request * num_requests
    
    input_cost = (total_input_tokens / 1000) * input_cost_per_1k
    output_cost = (total_output_tokens / 1000) * output_cost_per_1k
    
    total_cost = input_cost + output_cost
    return total_cost

cost = calculate_cost(input_tokens_per_request, output_tokens_per_request, num_requests, input_cost_per_1k, output_cost_per_1k)
print(f"The total cost for {num_requests} requests is: ${cost:.2f}")

The total cost for 1000 requests is: $1.72


Or roughly N2564.